In [4]:
import time
import requests
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output

# instantiating variables
titles = []
years = []
ratings = []
metascores = []
movieCounter = 1

# monitoring
start_time = time()
requests = 0

# iterate through # of pages
for _ in range(100):

    # get url
    url = get('https://www.imdb.com/search/title/?title_type=movie&genres=horror&start=' + str(movieCounter) + '&explore=title_type,genres&ref_=adv_nxt')

    # slow down loop
    sleep(randint(8,15))

    # monitor the requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # throw a warning for non-200 status codes
    if url.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, url.status_code))

    # break the loop if the number of requests is greater than expected
    if requests > 101:
        warn('Number of requests was greater than expected.')
        break

    # parse url
    soup = BeautifulSoup(url.content, 'html.parser')

    # select movies to scrape
    movie_elems = soup.find_all('div', class_='lister-item mode-advanced')
    
    # iterate through movies
    for movie_elem in movie_elems:
        if None in (movie_elem.h3.a.text, movie_elem.strong, movie_elem.span, movie_elem.find('span', class_ = 'metascore')):
            continue
        else:
            title = movie_elem.h3.a.text
            titles.append(title)
        
            year = ''.join(c for c in movie_elem.find('span', class_ = 'lister-item-year text-muted unbold').text if c.isdigit())
            years.append(int(year))
        
            rating = movie_elem.strong.text
            ratings.append(float(rating))
        
            metascore = movie_elem.find('span', class_ = 'metascore').text
            metascores.append(int(metascore))
    
    movieCounter += 50
    
    
# put data into pandas dataframe
movie_stats = pd.DataFrame({'movie': titles,
'year': years,
'user-rating': ratings,
'metascore': metascores
})

#print(movie_stats.info())
movie_stats['n_user-rating'] = movie_stats['user-rating'] * 10

# save data to csv file
movie_stats.to_csv('scraped_movie_data.csv')

Request:100; Frequency: 0.07713351251214996 requests/s
